<a href="https://colab.research.google.com/github/adityamavle/Domain_Specific_Financial_LLM/blob/master/FlanT5_finance_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://www.philschmid.de/fine-tune-flan-t5

In [2]:
# !pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
# # install git-fls for pushing model and logs to the hugging face hub
# !sudo apt-get install git-lfs --yes

  Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [2]:
!pip install datasets tensorboard

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

# Load dataset from the hub
dataset_id = "gbharti/finance-alpaca"
dataset = load_dataset(dataset_id,split = "train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
print(f"Train dataset size: {len(dataset['instruction'])}")

Train dataset size: 1000


In [4]:
df = dataset.to_pandas()
df.head()

,instruction,input,output,text
0,"For a car, what scams can be plotted with 0% f...",,The car deal makes money 3 ways. If you pay in...,
1,Why does it matter if a Central Bank has a neg...,,"That is kind of the point, one of the hopes is...",
2,Where should I be investing my money?,,"Pay off your debt. As you witnessed, no ""inve...",
3,Specifically when do options expire?,,"Equity options, at least those traded in the A...",
4,Negative Balance from Automatic Options Exerci...,,"Automatic exercisions can be extremely risky, ...",


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [7]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = dataset.map(lambda x: tokenizer(x["instruction"], truncation=True), batched=True, remove_columns=['input', 'output', 'text'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset.map(lambda x: tokenizer(x["output"], truncation=True), batched=True, remove_columns=['input','instruction', 'text'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Max source length: 40


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Max target length: 512


In [8]:
device_map = {"": 0}

In [9]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["instruction: " + inst for inst in sample['instruction']]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["output"], max_length=max_target_length, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["instruction", "output", "text","input"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [14]:
tokens = tokenized_dataset.to_pandas()

In [15]:
tokens.head()

,input_ids,attention_mask,labels
0,"[8033, 10, 242, 3, 9, 443, 6, 125, 13236, 7, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[37, 443, 1154, 656, 540, 220, 1155, 5, 156, 2..."
1,"[8033, 10, 1615, 405, 34, 1052, 3, 99, 3, 9, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[466, 19, 773, 13, 8, 500, 6, 80, 13, 8, 7511,..."
2,"[8033, 10, 2840, 225, 27, 36, 8448, 82, 540, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[5077, 326, 39, 2814, 5, 282, 25, 16903, 6, 15..."
3,"[8033, 10, 3, 22265, 116, 103, 931, 8982, 15, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[25943, 931, 6, 44, 709, 273, 18107, 16, 8, 79..."
4,"[8033, 10, 17141, 1528, 17904, 45, 19148, 1701...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19148, 23670, 159, 2865, 54, 36, 2033, 1020, ..."


In [18]:
del tokenizer
import gc
gc.collect()

165

In [10]:
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [24]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00


In [27]:
pip install accelerate -U

In [38]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [39]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=36fd8904a03f45e1b5a36ff709a5853d4a5d7dd76aad80aa93264fc27fbe26e4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [41]:
!pip install py7zr nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.3 MB/s eta 0:00:00


In [43]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [44]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [46]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# Hugging Face repository id
repository_id = "adityamavle/flan-t5-finance"
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=3,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    compute_metrics=compute_metrics,
    eval_dataset = tokenized_dataset
)


In [56]:
repository_id

'adityamavle/flan-t5-finance-alpaca'

In [47]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.081673,10.168100,1.634900,7.719700,9.190200,18.969000
2,No log,3.055386,10.835400,1.872900,8.236200,9.882200,18.937000
3,No log,3.045031,10.607400,1.820700,8.109000,9.680300,18.963000


TrainOutput(global_step=375, training_loss=3.2659619140625, metrics={'train_runtime': 859.0452, 'train_samples_per_second': 3.492, 'train_steps_per_second': 0.437, 'total_flos': 160490004480000.0, 'train_loss': 3.2659619140625, 'epoch': 3.0})

In [53]:
repository_id = "adityamavle/flan-t5-finance-alpaca"
tokenizer.save_pretrained(repository_id)

('adityamavle/flan-t5-finance-alpaca/tokenizer_config.json',
 'adityamavle/flan-t5-finance-alpaca/special_tokens_map.json',
 'adityamavle/flan-t5-finance-alpaca/tokenizer.json')

In [54]:
trainer.create_model_card()

In [50]:
!huggingface-cli login
model.push_to_hub()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-6597f8ac-3f25742d194593dc1ff92953;69978daf-74a7-4a81-8ce4-257097e9aa05)

Repository Not Found for url: https://huggingface.co/api/models/flan-t5-base-gbharti/finance-alpaca-v2.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [57]:
del trainer
del model
import gc
gc.collect()
gc.collect()

0

In [ ]:
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="/content/flan-t5-base-gbharti/finance-alpaca-v2", device=0)

In [58]:
base_model = AutoModelForCausalLM.from_pretrained(
    '/content/flan-t5-base-gbharti/finance-alpaca-v2',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

NameError: name 'AutoModelForCausalLM' is not defined

In [61]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

data = pd.read_csv('/content/data.csv')
data = data.head(100)

In [73]:
from transformers import pipeline
X = data["Sentence"]
y = data["Sentiment"]

def evaluate_lm_accuracy(lm, X_data, y_true):
    y_pred = []
    pipe = pipeline(task="text2text-generation", model="/content/flan-t5-base-gbharti/finance-alpaca-v2/checkpoint-375", tokenizer=tokenizer, max_length=1024)
    for prompt in X_data:
      prompt = f"Analyze the financial sentiment of the sentences, response will be one of the 3 labels = positive,negative,neutral: {prompt}"
      #print(prompt)
      ############inference code###########################
      result = pipe(f"instruction: {prompt} [/INST]")
      parts = result[0]['generated_text'].split("/INST")
      # answer = parts.lower()
      answer = parts[0]
      print(answer)
      sentiments = ["positive", "negative", "neutral"]
      # ans = None
      # for word in answer.lower().split():
      #     if word.strip(",.?!").lower() in sentiments:
      #         ans = word.strip(",.?!").lower()
      #     else:
      #         ans = 'neutral'
      #         break
      y_pred.append(answer)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

metrics_lm1 = evaluate_lm_accuracy("/content/flan-t5-base-gbharti/finance-alpaca-v2", X, y)
#metrics_lm2 = evaluate_lm_accuracy(lm_model_2, X, y)

print(f"Metrics of model 1: {metrics_lm1}")
#print(f"Metrics of model 2: {metrics_lm2}")

positive
negative
positive
positive
negative
positive
negative
negative
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
neutral
neutral
positive
positive
neutral
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
neutral
negative
positive
negative
negative
negative
negative
negative
negative
negative
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
negative
positive
negative
positive
neutral
negative
negative
positive
negative
positive
positive
negative
positive
positive
positive
negative
negative
negative
positive
negative
positive
positive
negative
positive
positive
negative
negative
positive
negative
negative
neutral
positive
negative
negative
positive
positive
positive
negative
negative
positive
negative
negative
Metrics of model 1: 0.49
